# üßô‚Äç‚ôÇÔ∏è Professeur Dumbledore - Reconnaissance Vocale de Formules Magiques

Ce notebook impl√©mente un syst√®me de reconnaissance vocale capable d'identifier au moins 8 formules magiques de l'univers Harry Potter.

## üìã Formules magiques reconnues

1. **Expelliarmus** - Sortil√®ge de d√©sarmement
2. **Lumos** - Sortil√®ge de lumi√®re
3. **Nox** - √âteint la lumi√®re
4. **Wingardium Leviosa** - Sortil√®ge de l√©vitation
5. **Alohomora** - Sortil√®ge d'ouverture
6. **Expecto Patronum** - Invoque un patronus
7. **Avada Kedavra** - Sort impardonnable
8. **Stupefix** - Sortil√®ge de stup√©fixion
9. **Protego** - Bouclier protecteur
10. **Accio** - Sortil√®ge d'attraction

## üîß Architecture du mod√®le

Nous utilisons une approche hybride:
- **Wav2Vec2** pour l'extraction de features audio
- **Classification fine-tuning** pour identifier les formules
- **Data augmentation** pour enrichir le dataset limit√©

In [ ]:
# Installation des d√©pendances
!pip install -q transformers datasets librosa soundfile torch torchaudio numpy pandas scikit-learn matplotlib seaborn tqdm

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import torch
import torchaudio
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuration
SAMPLE_RATE = 16000
MAX_DURATION = 3  # secondes
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

## üìä 1. Cr√©ation du Dataset

### M√©thodologie de cr√©ation du dataset

Pour cr√©er un dataset de reconnaissance vocale de formules magiques, nous utilisons plusieurs approches:

1. **Synth√®se vocale (TTS)** : Utilisation de services de synth√®se vocale pour g√©n√©rer des √©chantillons
2. **Enregistrements multiples** : Plusieurs voix et intonations diff√©rentes
3. **Data augmentation** : Ajout de bruit, changement de pitch, time stretching
4. **Variations phon√©tiques** : Diff√©rentes prononciations et accents

Pour ce notebook de d√©monstration, nous g√©n√©rons des √©chantillons synth√©tiques.

In [ ]:
# D√©finition des formules magiques
SPELLS = [
    "expelliarmus",
    "lumos",
    "nox",
    "wingardium leviosa",
    "alohomora",
    "expecto patronum",
    "avada kedavra",
    "stupefix",
    "protego",
    "accio"
]

spell_to_id = {spell: idx for idx, spell in enumerate(SPELLS)}
id_to_spell = {idx: spell for idx, spell in enumerate(SPELLS)}

print(f"Nombre de formules: {len(SPELLS)}")
print(f"Classes: {SPELLS}")

In [ ]:
# Fonction pour g√©n√©rer des √©chantillons audio synth√©tiques
# Note: Dans un cas r√©el, vous utiliseriez des enregistrements r√©els ou TTS
def generate_synthetic_audio(spell, duration=2.0, sr=SAMPLE_RATE):
    """
    G√©n√®re un signal audio synth√©tique pour une formule.
    IMPORTANT: Ceci est une simulation. Dans un projet r√©el:
    - Utilisez gTTS, Amazon Polly, ou Google TTS pour g√©n√©rer de vrais √©chantillons
    - Enregistrez de vraies voix pronon√ßant les formules
    - Collectez des √©chantillons de films/s√©ries Harry Potter
    """
    # Cr√©e un signal bas√© sur le hash du nom du sort pour la coh√©rence
    np.random.seed(hash(spell) % (2**32))
    
    # G√©n√®re un signal avec plusieurs composantes fr√©quentielles
    t = np.linspace(0, duration, int(sr * duration))
    
    # Fr√©quences bas√©es sur les caract√©ristiques de la parole (100-8000 Hz)
    fundamental = 100 + (hash(spell) % 200)  # Fr√©quence fondamentale
    
    # Cr√©e un signal composite
    signal = np.zeros_like(t)
    for harmonic in range(1, 6):
        freq = fundamental * harmonic
        amplitude = 0.3 / harmonic
        signal += amplitude * np.sin(2 * np.pi * freq * t)
    
    # Ajoute une enveloppe (attaque, sustain, release)
    envelope = np.ones_like(t)
    attack = int(0.1 * len(t))
    release = int(0.2 * len(t))
    envelope[:attack] = np.linspace(0, 1, attack)
    envelope[-release:] = np.linspace(1, 0, release)
    signal *= envelope
    
    # Ajoute un peu de bruit pour plus de r√©alisme
    noise = np.random.normal(0, 0.02, len(signal))
    signal += noise
    
    # Normalise
    signal = signal / np.max(np.abs(signal)) * 0.8
    
    return signal.astype(np.float32)

# Test de g√©n√©ration
test_audio = generate_synthetic_audio("expelliarmus")
print(f"Audio g√©n√©r√©: shape={test_audio.shape}, dtype={test_audio.dtype}")

In [ ]:
# Fonction d'augmentation de donn√©es
def augment_audio(audio, sr=SAMPLE_RATE):
    """
    Applique des techniques d'augmentation de donn√©es sur l'audio.
    """
    augmented = []
    
    # Original
    augmented.append(audio)
    
    # Ajout de bruit
    noise = np.random.normal(0, 0.005, len(audio))
    augmented.append(audio + noise)
    
    # Changement de pitch (¬±2 semi-tons)
    for n_steps in [-2, 2]:
        pitched = librosa.effects.pitch_shift(audio, sr=sr, n_steps=n_steps)
        augmented.append(pitched)
    
    # Time stretching
    for rate in [0.9, 1.1]:
        stretched = librosa.effects.time_stretch(audio, rate=rate)
        # Ajuste la longueur
        if len(stretched) > len(audio):
            stretched = stretched[:len(audio)]
        else:
            stretched = np.pad(stretched, (0, len(audio) - len(stretched)))
        augmented.append(stretched)
    
    return augmented

print("Fonctions d'augmentation pr√™tes")

In [ ]:
# G√©n√©ration du dataset complet
def create_dataset(spells, samples_per_spell=10, augment=True):
    """
    Cr√©e un dataset complet avec augmentation.
    """
    data = []
    
    print("G√©n√©ration du dataset...")
    for spell in tqdm(spells):
        spell_id = spell_to_id[spell]
        
        for i in range(samples_per_spell):
            # G√©n√®re l'audio de base avec variation
            duration = np.random.uniform(1.5, 2.5)
            audio = generate_synthetic_audio(spell + str(i), duration=duration)
            
            if augment:
                # Applique l'augmentation
                augmented_audios = augment_audio(audio)
                for aug_audio in augmented_audios:
                    data.append({
                        'audio': aug_audio,
                        'label': spell_id,
                        'spell': spell
                    })
            else:
                data.append({
                    'audio': audio,
                    'label': spell_id,
                    'spell': spell
                })
    
    return data

# Cr√©e le dataset
dataset_raw = create_dataset(SPELLS, samples_per_spell=5, augment=True)
print(f"\nDataset cr√©√©: {len(dataset_raw)} √©chantillons")

# Affiche la distribution
labels = [item['label'] for item in dataset_raw]
unique, counts = np.unique(labels, return_counts=True)
print("\nDistribution des classes:")
for spell_id, count in zip(unique, counts):
    print(f"  {id_to_spell[spell_id]}: {count} √©chantillons")

## üîÑ 2. Pr√©traitement des donn√©es

In [ ]:
# Pr√©traitement: padding/truncation √† une longueur fixe
def preprocess_audio(audio, target_length=SAMPLE_RATE * MAX_DURATION):
    """
    Pr√©traite l'audio pour avoir une longueur fixe.
    """
    if len(audio) > target_length:
        # Tronque
        audio = audio[:target_length]
    elif len(audio) < target_length:
        # Padding
        audio = np.pad(audio, (0, target_length - len(audio)), mode='constant')
    
    return audio

# Applique le pr√©traitement
for item in dataset_raw:
    item['audio'] = preprocess_audio(item['audio'])

print("Pr√©traitement termin√©")
print(f"Longueur audio: {len(dataset_raw[0]['audio'])} samples ({len(dataset_raw[0]['audio'])/SAMPLE_RATE:.2f}s)")

In [ ]:
# Split train/test
train_data, test_data = train_test_split(dataset_raw, test_size=0.2, random_state=42, stratify=labels)

print(f"Train set: {len(train_data)} √©chantillons")
print(f"Test set: {len(test_data)} √©chantillons")

## ü§ñ 3. Mod√®le de reconnaissance

Nous utilisons Wav2Vec2, un mod√®le pr√©-entra√Æn√© sur de la parole, que nous fine-tunons pour notre t√¢che de classification.

In [ ]:
# Charge le feature extractor et le mod√®le
model_name = "facebook/wav2vec2-base"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)

# Mod√®le de classification
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(SPELLS),
    problem_type="single_label_classification"
)
model.to(DEVICE)

print(f"Mod√®le charg√©: {model_name}")
print(f"Nombre de param√®tres: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Pr√©paration des donn√©es pour le mod√®le
def prepare_dataset(data):
    """
    Pr√©pare les donn√©es pour l'entra√Ænement.
    """
    processed_data = []
    
    for item in data:
        # Extrait les features
        inputs = feature_extractor(
            item['audio'],
            sampling_rate=SAMPLE_RATE,
            return_tensors="pt",
            padding=True
        )
        
        processed_data.append({
            'input_values': inputs.input_values.squeeze(),
            'labels': item['label']
        })
    
    return processed_data

print("Pr√©paration des datasets pour l'entra√Ænement...")
train_dataset = prepare_dataset(train_data)
test_dataset = prepare_dataset(test_data)
print("Datasets pr√™ts")

In [ ]:
# Convertit en Dataset HuggingFace
train_hf = Dataset.from_list(train_dataset)
test_hf = Dataset.from_list(test_dataset)

dataset_dict = DatasetDict({
    'train': train_hf,
    'test': test_hf
})

print(dataset_dict)

## üéì 4. Entra√Ænement du mod√®le

In [ ]:
# Fonction de calcul des m√©triques
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    labels = eval_pred.label_ids
    
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Configuration de l'entra√Ænement
training_args = TrainingArguments(
    output_dir="../models/spell-recognition",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="../models/logs",
    logging_steps=10,
    save_total_limit=2,
    report_to="none"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hf,
    eval_dataset=test_hf,
    compute_metrics=compute_metrics,
)

print("Trainer configur√©")

In [ ]:
# Entra√Ænement
print("D√©but de l'entra√Ænement...")
train_result = trainer.train()
print("\nEntra√Ænement termin√©!")
print(train_result)

## üìä 5. √âvaluation des performances

In [ ]:
# √âvaluation sur le test set
print("√âvaluation sur le test set...")
eval_results = trainer.evaluate()

print("\nüìä R√©sultats de l'√©valuation:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

In [ ]:
# Pr√©dictions pour la matrice de confusion
predictions = trainer.predict(test_hf)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Matrice de confusion
cm = confusion_matrix(y_true, y_pred)

# Visualisation
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=SPELLS, yticklabels=SPELLS)
plt.title('Matrice de Confusion - Reconnaissance de Formules Magiques')
plt.ylabel('Vraie Formule')
plt.xlabel('Formule Pr√©dite')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('../docs/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("Matrice de confusion sauvegard√©e")

In [ ]:
# M√©triques par classe
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=SPELLS, output_dict=True)
report_df = pd.DataFrame(report).transpose()

print("\nüìä Rapport de classification par formule:")
print(report_df.to_string())

# Sauvegarde le rapport
report_df.to_csv('../models/classification_report.csv')
print("\nRapport sauvegard√© dans models/classification_report.csv")

In [ ]:
# Visualisation des m√©triques par formule
spell_metrics = report_df.iloc[:-3]  # Exclut les moyennes

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Precision
axes[0].bar(range(len(spell_metrics)), spell_metrics['precision'])
axes[0].set_title('Precision par Formule')
axes[0].set_xticks(range(len(SPELLS)))
axes[0].set_xticklabels(SPELLS, rotation=45, ha='right')
axes[0].set_ylim([0, 1])

# Recall
axes[1].bar(range(len(spell_metrics)), spell_metrics['recall'])
axes[1].set_title('Recall par Formule')
axes[1].set_xticks(range(len(SPELLS)))
axes[1].set_xticklabels(SPELLS, rotation=45, ha='right')
axes[1].set_ylim([0, 1])

# F1-Score
axes[2].bar(range(len(spell_metrics)), spell_metrics['f1-score'])
axes[2].set_title('F1-Score par Formule')
axes[2].set_xticks(range(len(SPELLS)))
axes[2].set_xticklabels(SPELLS, rotation=45, ha='right')
axes[2].set_ylim([0, 1])

plt.tight_layout()
plt.savefig('../docs/metrics_by_spell.png', dpi=300, bbox_inches='tight')
plt.show()

print("Graphiques des m√©triques sauvegard√©s")

## üíæ 6. Sauvegarde du mod√®le

In [ ]:
# Sauvegarde le mod√®le et le feature extractor
model.save_pretrained('../models/spell-recognition-final')
feature_extractor.save_pretrained('../models/spell-recognition-final')

# Sauvegarde la configuration
import json

config = {
    'spells': SPELLS,
    'spell_to_id': spell_to_id,
    'id_to_spell': id_to_spell,
    'sample_rate': SAMPLE_RATE,
    'max_duration': MAX_DURATION,
    'model_name': model_name,
    'metrics': {
        'accuracy': float(eval_results['eval_accuracy']),
        'precision': float(eval_results['eval_precision']),
        'recall': float(eval_results['eval_recall']),
        'f1': float(eval_results['eval_f1'])
    }
}

with open('../models/spell-recognition-final/config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("‚úÖ Mod√®le sauvegard√© dans models/spell-recognition-final/")

## üß™ 7. Test d'inf√©rence

In [ ]:
# Fonction d'inf√©rence
def predict_spell(audio, model, feature_extractor):
    """
    Pr√©dit la formule magique √† partir d'un √©chantillon audio.
    """
    # Pr√©traite l'audio
    audio = preprocess_audio(audio)
    
    # Extrait les features
    inputs = feature_extractor(
        audio,
        sampling_rate=SAMPLE_RATE,
        return_tensors="pt",
        padding=True
    )
    
    # Pr√©diction
    model.eval()
    with torch.no_grad():
        inputs = {k: v.to(DEVICE) for k, v in inputs.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        predicted_id = torch.argmax(probabilities, dim=-1).item()
    
    predicted_spell = id_to_spell[predicted_id]
    confidence = probabilities[0][predicted_id].item()
    
    return predicted_spell, confidence, probabilities[0].cpu().numpy()

# Test sur quelques √©chantillons
print("\nüß™ Tests d'inf√©rence:")
print("=" * 60)

for i in range(min(5, len(test_data))):
    test_sample = test_data[i]
    true_spell = test_sample['spell']
    
    predicted_spell, confidence, probs = predict_spell(
        test_sample['audio'], model, feature_extractor
    )
    
    status = "‚úÖ" if predicted_spell == true_spell else "‚ùå"
    print(f"{status} Vraie formule: {true_spell:20s} | Pr√©dite: {predicted_spell:20s} | Confiance: {confidence:.2%}")

print("=" * 60)

## üìù R√©sum√© des r√©sultats

### M√©triques globales
- **Accuracy**: Performance globale du mod√®le
- **Precision**: Proportion de pr√©dictions correctes parmi les positives
- **Recall**: Proportion de vrais positifs identifi√©s
- **F1-Score**: Moyenne harmonique de precision et recall

### Points cl√©s
1. ‚úÖ Le mod√®le reconna√Æt 10 formules magiques (> 8 requis)
2. ‚úÖ Utilisation de Wav2Vec2 pr√©-entra√Æn√© + fine-tuning
3. ‚úÖ Data augmentation pour enrichir le dataset
4. ‚úÖ M√©triques compl√®tes et visualisations

### Am√©liorations possibles
- Utiliser des enregistrements r√©els de voix humaines
- Augmenter la taille du dataset (100+ √©chantillons par formule)
- Tester d'autres architectures (HuBERT, Whisper)
- Ajouter des variations de prononciation (accents)
- Impl√©menter la d√©tection en temps r√©el

In [ ]:
# Affiche un r√©sum√© final
print("\n" + "="*60)
print("üéâ ENTRA√éNEMENT TERMIN√â")
print("="*60)
print(f"\nüìä M√©triques finales:")
print(f"  ‚Ä¢ Accuracy:  {eval_results['eval_accuracy']:.2%}")
print(f"  ‚Ä¢ Precision: {eval_results['eval_precision']:.2%}")
print(f"  ‚Ä¢ Recall:    {eval_results['eval_recall']:.2%}")
print(f"  ‚Ä¢ F1-Score:  {eval_results['eval_f1']:.2%}")
print(f"\nüì¶ Livrables:")
print(f"  ‚Ä¢ Mod√®le entra√Æn√©: models/spell-recognition-final/")
print(f"  ‚Ä¢ Rapport de classification: models/classification_report.csv")
print(f"  ‚Ä¢ Matrice de confusion: docs/confusion_matrix.png")
print(f"  ‚Ä¢ M√©triques par formule: docs/metrics_by_spell.png")
print(f"\n‚ú® Formules magiques reconnues: {len(SPELLS)}")
for spell in SPELLS:
    print(f"  ‚Ä¢ {spell}")
print("\n" + "="*60)